# Pujol-like shear Calibration

In [ ]:
%pylab inline

In [ ]:
import galsim

In [ ]:
def generate_gal(i):
  #cat
  cat = galsim.COSMOSCatalog(sample="25.2")
  gal = cat.makeGalaxy(i, gal_type='parametric')
  gal_mag = cat.param_cat['mag_auto'][cat.orig_index[i]]  
  ##cfis photometry
  mag_zp = 32
  gal_flux = 10**(-(gal_mag-mag_zp)/2.5)
  gal = gal.withFlux(gal_flux)
  return gal


In [ ]:
def noise_image(size=51,sky_level=400):
  rng = np.random.RandomState(1)
  noise=rng.normal(loc=sky_level,scale=np.sqrt(sky_level),size=[size,size])
  return noise 

In [ ]:
def galsimator(image,flux=1.0,scale=0.1):
  galsim_img = galsim.InterpolatedImage(image,x_interpolant='lanczos14')
  return galsim_img  

In [ ]:
def pujol_sims(gal_model,psf_model,step=0.02):
  gal1p = gal.shear(step,0)
  gal2p = gal.shear(0,step)
  
  #noshear
  obsns = galsim.Convolve([gal,psf_model])
  #1p
  obs1p = galsim.Convolve([gal1p,psf_model])
  #2p
  obs2p = galsim.Convolve([gal2p,psf_model])
    
  pujobsdict = {
    'noshear':obsns,
    '1p':obs1p,
    '2p':obs2p,  
  }
  
  return  pujobsdict

In [ ]:
def apply_noise(pujobsdict,noise):
  for key in pujobsdict.keys():
    pujobsdict[key] += noise
  return pujobsdict

In [ ]:
def measure_ellipticities(pujobsdict,method):
  resdict = {}
  for key in pujobsdict.keys():
    resdict += {key: method(pujobsdict[key])}
  return resdict

In [ ]:
def get_response(resdict):

  #noshear
  g0s = np.array(resdict['noshear'])
  
  #shear
  g1p = np.array(resdict['1p'])
  g2p = np.array(resdict['2p'])   
  
  R11 = (g1p[0]-g0s[0])/(2*step)
  R21 = (g1p[1]-g0s[1])/(2*step) 
  R12 = (g2p[0]-g0s[0])/(2*step)
  R22 = (g2p[1]-g0s[1])/(2*step)
  
  R = np.array(
    [[R11,R12],
     [R21,R22]])

  ellip_dict = {
    'noshear':g0s,
    '1p':g1p,
    '1m':g1m,
    '2p':g2p,
    '2m':g2m,  
  } 
      
  return ellip_dict, R